In [37]:
import sys
import wget
import newspaper
from newspaper import Article
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
import lxml
import xmltodict
import collections
from urllib import request
from collections import OrderedDict
from urllib.error import HTTPError, URLError
from urllib.parse import urlparse
import sys
import ast
import time
from pandarallel import pandarallel
import requests
import datefinder
from datetime import datetime
from tqdm import tqdm
import psutil
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from multiprocessing import Pool
import random

In [42]:
#PLEASE EDIT THIS CELL!

#Universal parameters. Always needs to be changed
news_outlet = "mercopress"
country = "south_america"
max_click_SHOW_MORE = 500
host = "https://es.mercopress.com/"
date_format = "%Y-%m-%d"
relevant_path = f'^{host}.*$'
urls = [host + 'agricultura',host + 'economia',host + 'energia', host + 'entretenimiento', host + 'medio-ambiente', host + 'pesca', host + 'ciencia-y-salud', host + 'inversiones', host + 'bienes-raices', host + 'turismo']
csv_name = country+'_'+news_outlet+'.csv'
types = ['page', 'click_more','scroll_down']
type_of_page = types[0]
extract_text_sleep = random.randint(4,6)
extract_urls_sleep = random.randint(2,4)





# If page type is click_more or scroll_down i.e. can get more article by clicking a button with text 'click more' or simply scrolling down
xpath_for_link = "//h2[contains(@class, 'Card-Title Title Title')]//following::a"
xpath_for_click_more = "//svg[contains(@class, 'Pagination-Icon')]"





# If page type is page
target_tag = 'h2'
target_tag_class = 'title'




## This has been newly added! ## 
page_url_suffices = ["/", "?p="]
page_url_suffix = page_url_suffices[1]






if type_of_page == 'page':
    urls = [url for url in urls] #   + 'page/'
pages_each_category = [] #The total number of pages for each section. So if policiaca is 1..92, write 92, not 93.
if pages_each_category:
    total_pages = pages_each_category
else:
    total_pages = 300

In [43]:
def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [44]:
def retrieve_links_from_list(url,scroll_down=False):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(executable_path="chromedriver", options=chrome_options, )
  driver.get(url)
  
  count = 0

  news_urls = set()

  prev_length = 0
  while count <= max_click_SHOW_MORE:
      if not scroll_down:
          try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, xpath_for_click_more))
            )
          except TimeoutException as e:
            break
        
    
      try:
          all_links = driver.find_elements(By.XPATH, xpath_for_link)
          if not scroll_down:
            click_more = driver.find_element(By.XPATH, xpath_for_click_more)
          curr_length = len(all_links)
          print(f'{curr_length} links in current page')
          print(f'{prev_length} links in previous page')
          if count > 0:
            if curr_length == prev_length or (not scroll_down and not click_more):
              break
          
          prev_length = curr_length

          if not scroll_down:
            click_more.click();
            print(f"Button clicked {count+1} times", )
          else:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            print(f"Scrolled down {count+1} times")
          
          time.sleep(3)
            
          count += 1
        
          
          
      except TimeoutException:
          break

  time.sleep(2)

  all_links = driver.find_elements(By.XPATH, xpath_for_link)

  for link in all_links:
        if isinstance(link.get_attribute('href'), str):
          if re.match(relevant_path, link.get_attribute('href')):
            news_urls.add(link.get_attribute('href'))

  driver.quit()
  print('*' * 20)
  return list(news_urls)


def extract_urls(url):    
    rows = set()
    soup = getSoup(url)
    if soup:
      all_elems = None
      if target_tag_class != '':
        all_elems = soup.find_all(target_tag, {'class':target_tag_class})
      else:
        all_elems = soup.find_all(target_tag)
      
      for d in all_elems:
        all_links = d.find_all('a', href=True)
        
        for l in all_links:
            if not re.match('^http.*', l['href']):
              l['href'] = host + l['href']
              
            if re.match(relevant_path, l['href']):
              rows.add(l['href'])
                
    
    if rows:
      return list(rows)
    else:
      return None

def extract_urls_from_df(df):
    links = df['page'].map(lambda x: extract_urls(x))
    df['link'] = links
    return df

In [45]:
pages = []

def get_zipped_urls(url, total_pages):
    page_numbers = [page_url_suffix + str(i) for i in range(1,total_pages)]
    url_multiple = [url] * total_pages
    return [''.join(x) for x in zip(url_multiple,page_numbers)]

if type_of_page == 'page':
    for ind, url in enumerate(urls):
        if isinstance(total_pages, int):
            pages += get_zipped_urls(url, total_pages)
        else:
            pages_for_category = total_pages[ind] + 1
            pages += get_zipped_urls(url, pages_for_category)
            
elif type_of_page == 'click_more' or type_of_page == 'scroll_down':
    for url in urls:
        pages += retrieve_links_from_list(url, True if type_of_page == 'scroll_down' else False)

        
df = pd.DataFrame(pages, columns = ['page'])
df

,page
0,https://es.mercopress.com/economia?p=100
1,https://es.mercopress.com/economia?p=101
2,https://es.mercopress.com/economia?p=102
3,https://es.mercopress.com/economia?p=103
4,https://es.mercopress.com/economia?p=104
...,...
195,https://es.mercopress.com/economia?p=295
196,https://es.mercopress.com/economia?p=296
197,https://es.mercopress.com/economia?p=297
198,https://es.mercopress.com/economia?p=298


In [46]:
def getSoup(url):
    """
    Return a soup object of the URL
    """
    try:
        req = request.Request(url, headers={'User-Agent' : "Chrome"}) 
        con = request.urlopen(req)
        time.sleep(extract_urls_sleep)
        html = con.read()

    except HTTPError as e:
        print(e)
        return None
        
    except URLError as e:
        print('The server could not be found')
        return None

    except Exception as e:
      print(e)
      return None

    
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [47]:
def extract_text(url):
    text, title, date = None, None, None
    try:
        result = [country, news_outlet] + [''] * 3 
        article = Article(url, keep_article_html=False)
        article.download()
        time.sleep(extract_text_sleep)
        article.parse()
        text = article.text
        
        if text:
            text_copy = text
            title = article.title
            date = article.publish_date
            if date:
                date = date.strftime(date_format)
            else:
                matches = datefinder.find_dates(text_copy)
                most_recent_datetime = sorted(matches)[-1]
                date = most_recent_datetime.strftime(date_format)
    finally:
      if title:
        result[2] = title
      if date:
        result[3] = date
      if text:
        result[4] = text
      return result 


def extract_text_from_df(df):
    content = df['link'].map(lambda x: extract_text(x))
    df['content'] = content
    return df

In [48]:
df

,page
0,https://es.mercopress.com/economia?p=100
1,https://es.mercopress.com/economia?p=101
2,https://es.mercopress.com/economia?p=102
3,https://es.mercopress.com/economia?p=103
4,https://es.mercopress.com/economia?p=104
...,...
195,https://es.mercopress.com/economia?p=295
196,https://es.mercopress.com/economia?p=296
197,https://es.mercopress.com/economia?p=297
198,https://es.mercopress.com/economia?p=298


In [49]:
def parallel_work(df, method_to_run, target, source):
  global workers
  while workers>0:
    try:
      df[target] = df[source].parallel_apply(method_to_run)
    except Exception as e:  
      raise e
    break

  if workers == 0:
    print('Error during parallel operation. Could not extract text')

  return df

def get_parallel_operation_results(divided_dfs, method_to_run, target, source):
  res = []
  for df in divided_dfs:
    try:
      temp_df = parallel_work(df, method_to_run, target, source)
      if temp_df[target].isnull().all():
        print('Could not retrieve any URLS')
        print('Something is wrong with the target_tag and target_tag_class variables. Please modify')
        return []
      res.append(temp_df)
    except Exception:
      continue
  df_result = pd.concat(res)
  return df_result

def partition_df(df):
  global articles_per_parallel_operation
  divided_dfs = []
  start = 0
  while start < len(df):
    divided_dfs.append(df[start:start+articles_per_parallel_operation])
    start += articles_per_parallel_operation
  return divided_dfs

In [50]:
if type_of_page == 'page':    
    start = 0
    limit = 180
    total_time_start = time.time()
    results = []
    
    while start < len(df):
        start_time = time.time()
        results.append(parallelize_dataframe(df[start:start+limit], extract_urls_from_df, 90))
        end_time = time.time()
        print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
        print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
        start+=limit
    
    df = pd.concat(results)
    total_time_end = time.time()
    print(f'total time taken: {round(total_time_end - total_time_start,2)} second')

else:
    df = df.rename(columns={"page": "link"})

Batch of data of row range 0-180 complete in 34.22 seconds
90.0% complete
Batch of data of row range 180-360 complete in 6.79 seconds
100% complete
total time taken: 41.02 second


In [51]:
df

,page,link
0,https://es.mercopress.com/economia?p=100,"[https://es.mercopress.com/2020/04/30/boeing-cancela-acuerdo-por-us-4.200-millones-y-embraer-comienza-proceso-de-arbitraje, https://es.mercopress.com/2020/04/28/argentina-suspende-venta-de-pasajes-de-vuelos-comerciales-hasta-setiembre, https://es.mercopress.com/2020/04/27/ni-gasolina-barata-queda-en-venezuela-2-dolares-el-litro-y-atenciones-para-el-personal-de-servicio, https://es.mercopress.com/2020/04/28/magnate-naviero-envio-150.000-barriles-de-gasolina-a-venezuela, https://es.mercopress.com/2020/04/29/oit-preve-perdida-de-300-millones-de-empleos-en-america-latina-debido-al-covid-19, ht..."
1,https://es.mercopress.com/economia?p=101,"[https://es.mercopress.com/2020/04/23/el-parana-marca-la-mayor-bajante-historica-y-su-menor-caudal-frente-a-entre-rios, https://es.mercopress.com/2020/04/25/gobierno-de-falklands-comprara-la-lana-sin-vender-71-de-la-zafra-actual, https://es.mercopress.com/2020/04/24/argentina-planea-un-barril-criollo-para-intentar-salvar-la-industria-del-petroleo-de-su-desplome-global, https://es.mercopress.com/2020/04/25/industria-pesquera-de-falklands-sufre-el-impacto-de-la-pandemia-de-covid-19, https://es.mercopress.com/2020/04/25/empresas-de-pesca-de-falklands-ante-una-demanda-incierta-precios-a-la-baj..."
2,https://es.mercopress.com/economia?p=102,"[https://es.mercopress.com/2020/04/20/falklands-anuncia-medidas-en-apoyo-al-empleo-privado-y-a-las-empresas, https://es.mercopress.com/2020/04/21/el-petroleo-en-estados-unidos-a-precio-negativo-por-excedentes-falta-de-depositos-y-desplome-de-consumo, https://es.mercopress.com/2020/04/21/precios-mayoristas-en-argentina-suben-1-en-marzo-y-costo-de-construccion-1-4, https://es.mercopress.com/2020/04/18/brasil-prepara-paquete-de-us-8.000-millones-para-sector-turismo-y-100.000-puestos-de-trabajo, https://es.mercopress.com/2020/04/21/grupos-de-bonistas-rechazan-propuesta-argentina-pero-hay-volun..."
3,https://es.mercopress.com/economia?p=103,"[https://es.mercopress.com/2020/04/16/iata-anuncia-que-no-habra-reembolso-del-pago-por-boletos-aereos-serian-unos-us-35.000-millones, https://es.mercopress.com/2020/04/16/argentina-solicita-a-la-sec-emitir-bonos-por-us-50.500-millones-tambien-pidio-postergar-pagos-al-club-de-paris, https://es.mercopress.com/2020/04/17/en-ciudades-de-eeuu-salen-a-protestar-la-cuarentena-y-piden-reabrir-la-economia, https://es.mercopress.com/2020/04/17/argentina-presento-propuesta-a-bonistas-privados-no-habria-pagos-hasta-2023-y-una-quita-del-62-en-los-intereses, https://es.mercopress.com/2020/04/17/record-d..."
4,https://es.mercopress.com/economia?p=104,"[https://es.mercopress.com/2020/04/13/latam-suspende-todas-sus-operaciones-internacionales-hasta-el-30-de-abril, https://es.mercopress.com/2020/04/09/en-argentina-actividad-industrial-cayo-0.8-y-la-construccion-22-1-interanual-en-febrero, https://es.mercopress.com/2020/04/13/banco-mundial-estima-que-la-economia-regional-se-contraera-4-6-en-2020-pero-rebotara-2-3-en-2021, https://es.mercopress.com/2020/04/14/fmi-economia-de-uruguay-caera-3-en-2020-la-de-latinoamerica-5-2, https://es.mercopress.com/2020/04/09/bernanke-pronostica-que-economia-de-eeuu-caera-30-este-trimestre, https://es.mercop..."
...,...,...
195,https://es.mercopress.com/economia?p=295,"[https://es.mercopress.com/2017/01/17/para-fmi-hay-menor-expectativa-de-recuperacion-a-corto-plazo-en-argentina-y-brasil, https://es.mercopress.com/2017/01/19/confirman-ortodoxia-financiera-en-argentina-remueven-al-presidente-del-banco-nacion, https://es.mercopress.com/2017/01/20/falklands-analiza-y-toma-recaudos-tras-el-discurso-sobre-brexit-de-theresa-may, https://es.mercopress.com/2017/01/17/rolls-royce-pagara-us-805-millones-para-cerrar-casos-de-corrupcion, https://es.mercopress.com/2017/01/17/maduro-desconoce-al-legislativo-y-admitio-que-2016-fue-el-ano-mas-duro-y-largo, https://es.me..."
196,https://es.mercopress.com/economia?p=296,"[https://es.mercopress.com/2017/01/10/bancos-de-ee.uu.-opti

In [52]:
df.to_csv('links_'+csv_name)

In [53]:
print(len(df))
df = df[df['link'].notna()]
df = df[(df['link'].str.len() != 0)]
print(len(df))

200
200


In [54]:
if type_of_page == 'page':
    df = df.explode('link')
    
df

,page,link
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/30/boeing-cancela-acuerdo-por-us-4.200-millones-y-embraer-comienza-proceso-de-arbitraje
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/28/argentina-suspende-venta-de-pasajes-de-vuelos-comerciales-hasta-setiembre
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/27/ni-gasolina-barata-queda-en-venezuela-2-dolares-el-litro-y-atenciones-para-el-personal-de-servicio
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/28/magnate-naviero-envio-150.000-barriles-de-gasolina-a-venezuela
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/29/oit-preve-perdida-de-300-millones-de-empleos-en-america-latina-debido-al-covid-19
...,...,...
199,https://es.mercopress.com/economia?p=299,https://es.mercopress.com/2016/12/27/balanza-comercial-de-brasil-con-record-de-us-45.000mn
199,https://es.mercopress.com/economia?p=299,https://es.mercopress.com/2016/12/24/un-terreno-para-ong-de-lula-fue-adquirido-por-odebrecht
199,https://es.mercopress.com/economia?p=299,https://es.mercopress.com/2016/12/24/italia-vota-rescate-con-fondos-publicos-para-bancos-con-dificultades
199,https://es.mercopress.com/economia?p=299,https://es.mercopress.com/2016/12/24/temer-anuncia-reforma-laboral-y-flexibilizar-jornadas-de-trabajo-para-combatir-desempleo


In [55]:
print(len(df))
df = df.drop_duplicates(subset='link')
print(len(df))

2000
2000


In [56]:
#Create a links checkpoint csv
df.to_csv('links_'+csv_name)

In [57]:
if 'df' not in locals() or df is None or 'link' not in df:
    if(os.path.exists('links_'+csv_name)):
        df = pd.read_csv('links_'+csv_name)

In [58]:
df

,page,link
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/30/boeing-cancela-acuerdo-por-us-4.200-millones-y-embraer-comienza-proceso-de-arbitraje
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/28/argentina-suspende-venta-de-pasajes-de-vuelos-comerciales-hasta-setiembre
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/27/ni-gasolina-barata-queda-en-venezuela-2-dolares-el-litro-y-atenciones-para-el-personal-de-servicio
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/28/magnate-naviero-envio-150.000-barriles-de-gasolina-a-venezuela
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/29/oit-preve-perdida-de-300-millones-de-empleos-en-america-latina-debido-al-covid-19
...,...,...
199,https://es.mercopress.com/economia?p=299,https://es.mercopress.com/2016/12/27/balanza-comercial-de-brasil-con-record-de-us-45.000mn
199,https://es.mercopress.com/economia?p=299,https://es.mercopress.com/2016/12/24/un-terreno-para-ong-de-lula-fue-adquirido-por-odebrecht
199,https://es.mercopress.com/economia?p=299,https://es.mercopress.com/2016/12/24/italia-vota-rescate-con-fondos-publicos-para-bancos-con-dificultades
199,https://es.mercopress.com/economia?p=299,https://es.mercopress.com/2016/12/24/temer-anuncia-reforma-laboral-y-flexibilizar-jornadas-de-trabajo-para-combatir-desempleo


In [59]:
output_path= 'unprocessed_' + csv_name
start = 0
limit = 360

res = []
while start < len(df):
    start_time = time.time()
    demo_df = df[start:start+limit].copy()
    test_df = parallelize_dataframe(demo_df, extract_text_from_df, 90)
    res.append(test_df)
    test_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
    
    test_df['text_set'] = test_df['content'].map(lambda x: True if x[4] != '' else False)    
    no_of_text_retrieved = test_df.text_set.sum()
    print(f'{no_of_text_retrieved} / {len(test_df)} text retrieved')
    
    end_time = time.time()
    print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
    print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
    start+=limit

df = pd.concat(res)

360 / 360 text retrieved
Batch of data of row range 0-360 complete in 29.47 seconds
18.0% complete
360 / 360 text retrieved
Batch of data of row range 360-720 complete in 30.19 seconds
36.0% complete
360 / 360 text retrieved
Batch of data of row range 720-1080 complete in 29.49 seconds
54.0% complete
360 / 360 text retrieved
Batch of data of row range 1080-1440 complete in 28.34 seconds
72.0% complete
360 / 360 text retrieved
Batch of data of row range 1440-1800 complete in 28.02 seconds
90.0% complete
200 / 200 text retrieved
Batch of data of row range 1800-2160 complete in 20.2 seconds
100% complete


In [60]:
output_path= 'unprocessed_' + csv_name
if 'df' not in locals() or df is None or 'content' not in df:
    if(os.path.exists(output_path)):
        df = pd.read_csv(output_path, on_bad_lines='skip', converters={'content': pd.eval})




In [61]:
df[['country', 'news_outlet', 'title', 'date', 'text']] = pd.DataFrame(df.content.tolist(), index= df.index)     
df

,page,link,content,text_set,country,news_outlet,title,date,text
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/30/boeing-cancela-acuerdo-por-us-4.200-millones-y-embraer-comienza-proceso-de-arbitraje,"[south_america, mercopress, Boeing cancela acuerdo por US$ 4.200 millones y Embraer comienza proceso de arbitraje, 2020-04-30, Boeing cancela acuerdo por US$ 4.200 millones y Embraer comienza proceso de arbitraje\n\nLas acciones de Embraer se hundieron hasta un 16% en Sao Paulo a un mínimo de más de 8 años debido a las noticias de la cancelación\n\nEmbraer esperaba vender el 80% de su rentable unidad de aviación comercial a Boeing y beneficiarse de ese poder de marketing para aumentar las ventas\n\nLa brasileña Embraer SA dijo que había comenzado un proceso de arbitraje contra Boeing Co, l...",True,south_america,mercopress,Boeing cancela acuerdo por US$ 4.200 millones y Embraer comienza proceso de arbitraje,2020-04-30,"Boeing cancela acuerdo por US$ 4.200 millones y Embraer comienza proceso de arbitraje\n\nLas acciones de Embraer se hundieron hasta un 16% en Sao Paulo a un mínimo de más de 8 años debido a las noticias de la cancelación\n\nEmbraer esperaba vender el 80% de su rentable unidad de aviación comercial a Boeing y beneficiarse de ese poder de marketing para aumentar las ventas\n\nLa brasileña Embraer SA dijo que había comenzado un proceso de arbitraje contra Boeing Co, luego que el fabricante estadounidense de aviones cancelara abruptamente durante el fin de semana un acuerdo de US$ 4.200 millon..."
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/28/argentina-suspende-venta-de-pasajes-de-vuelos-comerciales-hasta-setiembre,"[south_america, mercopress, Argentina suspende venta de pasajes de vuelos comerciales hasta setiembre, 2020-04-28, Argentina suspende venta de pasajes de vuelos comerciales hasta setiembre\n\nEl objetivo del decreto es evitar que las aerolíneas emitan boletos que no hayan sido aprobados por el Gobierno del presidente Alberto Fernández.\n\nArgentina restringió el lunes la venta de pasajes para vuelos comerciales hasta septiembre, una nueva medida del Gobierno para detener la pandemia de coronavirus en el país que despertó críticas en la industria de la aviación, que señaló que la decisión t...",True,south_america,mercopress,Argentina suspende venta de pasajes de vuelos comerciales hasta setiembre,2020-04-28,"Argentina suspende venta de pasajes de vuelos comerciales hasta setiembre\n\nEl objetivo del decreto es evitar que las aerolíneas emitan boletos que no hayan sido aprobados por el Gobierno del presidente Alberto Fernández.\n\nArgentina restringió el lunes la venta de pasajes para vuelos comerciales hasta septiembre, una nueva medida del Gobierno para detener la pandemia de coronavirus en el país que despertó críticas en la industria de la aviación, que señaló que la decisión traerá más presión sobre compañías y aeropuertos.\n\nEl decreto publicado en el Boletín Oficial indica que no se pod..."
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/27/ni-gasolina-barata-queda-en-venezuela-2-dolares-el-litro-y-atenciones-para-el-personal-de-servicio,"[south_america, mercopress, Ni gasolina barata queda en Venezuela: 2 dólares el litro y atenciones para el personal de servicio, 2020-04-27, Ni gasolina barata queda en Venezuela: 2 dólares el litro y atenciones para el personal de servicio\n\nMuchos automovilistas ofrecen a los trabajadores de las estaciones de servicio propinas de bocadillos o unos pocos cientos de bolívares, en lugar de pago (Foto AP)\n\nEn el estado de Bolívar las autoridades han cerrado al público la mayoría de la estaciones por la cuarentena en vigencia desde mediados de marzo\n\nAlgunos venezolanos están diciendo qu...",True,south_america,mercopress,Ni gasolina barata queda en Venezuela: 2 dólares el litro y atenciones para el personal de servicio,2020-04-27,"Ni gasolina barata queda en Venezuela: 2 dól

In [62]:
print(len(df))
df = df[df['text'] != '']
print(len(df))
df

2000
2000


,page,link,content,text_set,country,news_outlet,title,date,text
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/30/boeing-cancela-acuerdo-por-us-4.200-millones-y-embraer-comienza-proceso-de-arbitraje,"[south_america, mercopress, Boeing cancela acuerdo por US$ 4.200 millones y Embraer comienza proceso de arbitraje, 2020-04-30, Boeing cancela acuerdo por US$ 4.200 millones y Embraer comienza proceso de arbitraje\n\nLas acciones de Embraer se hundieron hasta un 16% en Sao Paulo a un mínimo de más de 8 años debido a las noticias de la cancelación\n\nEmbraer esperaba vender el 80% de su rentable unidad de aviación comercial a Boeing y beneficiarse de ese poder de marketing para aumentar las ventas\n\nLa brasileña Embraer SA dijo que había comenzado un proceso de arbitraje contra Boeing Co, l...",True,south_america,mercopress,Boeing cancela acuerdo por US$ 4.200 millones y Embraer comienza proceso de arbitraje,2020-04-30,"Boeing cancela acuerdo por US$ 4.200 millones y Embraer comienza proceso de arbitraje\n\nLas acciones de Embraer se hundieron hasta un 16% en Sao Paulo a un mínimo de más de 8 años debido a las noticias de la cancelación\n\nEmbraer esperaba vender el 80% de su rentable unidad de aviación comercial a Boeing y beneficiarse de ese poder de marketing para aumentar las ventas\n\nLa brasileña Embraer SA dijo que había comenzado un proceso de arbitraje contra Boeing Co, luego que el fabricante estadounidense de aviones cancelara abruptamente durante el fin de semana un acuerdo de US$ 4.200 millon..."
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/28/argentina-suspende-venta-de-pasajes-de-vuelos-comerciales-hasta-setiembre,"[south_america, mercopress, Argentina suspende venta de pasajes de vuelos comerciales hasta setiembre, 2020-04-28, Argentina suspende venta de pasajes de vuelos comerciales hasta setiembre\n\nEl objetivo del decreto es evitar que las aerolíneas emitan boletos que no hayan sido aprobados por el Gobierno del presidente Alberto Fernández.\n\nArgentina restringió el lunes la venta de pasajes para vuelos comerciales hasta septiembre, una nueva medida del Gobierno para detener la pandemia de coronavirus en el país que despertó críticas en la industria de la aviación, que señaló que la decisión t...",True,south_america,mercopress,Argentina suspende venta de pasajes de vuelos comerciales hasta setiembre,2020-04-28,"Argentina suspende venta de pasajes de vuelos comerciales hasta setiembre\n\nEl objetivo del decreto es evitar que las aerolíneas emitan boletos que no hayan sido aprobados por el Gobierno del presidente Alberto Fernández.\n\nArgentina restringió el lunes la venta de pasajes para vuelos comerciales hasta septiembre, una nueva medida del Gobierno para detener la pandemia de coronavirus en el país que despertó críticas en la industria de la aviación, que señaló que la decisión traerá más presión sobre compañías y aeropuertos.\n\nEl decreto publicado en el Boletín Oficial indica que no se pod..."
0,https://es.mercopress.com/economia?p=100,https://es.mercopress.com/2020/04/27/ni-gasolina-barata-queda-en-venezuela-2-dolares-el-litro-y-atenciones-para-el-personal-de-servicio,"[south_america, mercopress, Ni gasolina barata queda en Venezuela: 2 dólares el litro y atenciones para el personal de servicio, 2020-04-27, Ni gasolina barata queda en Venezuela: 2 dólares el litro y atenciones para el personal de servicio\n\nMuchos automovilistas ofrecen a los trabajadores de las estaciones de servicio propinas de bocadillos o unos pocos cientos de bolívares, en lugar de pago (Foto AP)\n\nEn el estado de Bolívar las autoridades han cerrado al público la mayoría de la estaciones por la cuarentena en vigencia desde mediados de marzo\n\nAlgunos venezolanos están diciendo qu...",True,south_america,mercopress,Ni gasolina barata queda en Venezuela: 2 dólares el litro y atenciones para el personal de servicio,2020-04-27,"Ni gasolina barata queda en Venezuela: 2 dól

In [63]:
if 'page' in df:
  df = df.drop(['page'], axis=1)
if 'content' in df:
  df = df.drop(['content'], axis=1)
if 'Unnamed: 0' in df:
    df = df.drop(['Unnamed: 0'], axis=1)
if 'Unnamed: 0.1' in df:
    df = df.drop(['Unnamed: 0.1'], axis=1)
if 'text_set' in df:
    df = df.drop(['text_set'], axis=1)

In [64]:
df

,link,country,news_outlet,title,date,text
0,https://es.mercopress.com/2020/04/30/boeing-cancela-acuerdo-por-us-4.200-millones-y-embraer-comienza-proceso-de-arbitraje,south_america,mercopress,Boeing cancela acuerdo por US$ 4.200 millones y Embraer comienza proceso de arbitraje,2020-04-30,"Boeing cancela acuerdo por US$ 4.200 millones y Embraer comienza proceso de arbitraje\n\nLas acciones de Embraer se hundieron hasta un 16% en Sao Paulo a un mínimo de más de 8 años debido a las noticias de la cancelación\n\nEmbraer esperaba vender el 80% de su rentable unidad de aviación comercial a Boeing y beneficiarse de ese poder de marketing para aumentar las ventas\n\nLa brasileña Embraer SA dijo que había comenzado un proceso de arbitraje contra Boeing Co, luego que el fabricante estadounidense de aviones cancelara abruptamente durante el fin de semana un acuerdo de US$ 4.200 millon..."
0,https://es.mercopress.com/2020/04/28/argentina-suspende-venta-de-pasajes-de-vuelos-comerciales-hasta-setiembre,south_america,mercopress,Argentina suspende venta de pasajes de vuelos comerciales hasta setiembre,2020-04-28,"Argentina suspende venta de pasajes de vuelos comerciales hasta setiembre\n\nEl objetivo del decreto es evitar que las aerolíneas emitan boletos que no hayan sido aprobados por el Gobierno del presidente Alberto Fernández.\n\nArgentina restringió el lunes la venta de pasajes para vuelos comerciales hasta septiembre, una nueva medida del Gobierno para detener la pandemia de coronavirus en el país que despertó críticas en la industria de la aviación, que señaló que la decisión traerá más presión sobre compañías y aeropuertos.\n\nEl decreto publicado en el Boletín Oficial indica que no se pod..."
0,https://es.mercopress.com/2020/04/27/ni-gasolina-barata-queda-en-venezuela-2-dolares-el-litro-y-atenciones-para-el-personal-de-servicio,south_america,mercopress,Ni gasolina barata queda en Venezuela: 2 dólares el litro y atenciones para el personal de servicio,2020-04-27,"Ni gasolina barata queda en Venezuela: 2 dólares el litro y atenciones para el personal de servicio\n\nMuchos automovilistas ofrecen a los trabajadores de las estaciones de servicio propinas de bocadillos o unos pocos cientos de bolívares, en lugar de pago (Foto AP)\n\nEn el estado de Bolívar las autoridades han cerrado al público la mayoría de la estaciones por la cuarentena en vigencia desde mediados de marzo\n\nAlgunos venezolanos están diciendo que pagaron esta semana más de 2 dólares por un litro de gasolina en medio de la escasez de combustible que sufre la nación y que coloca tal pr..."
0,https://es.mercopress.com/2020/04/28/magnate-naviero-envio-150.000-barriles-de-gasolina-a-venezuela,south_america,mercopress,Magnate naviero envió 150.000 barriles de gasolina a Venezuela,2020-04-28,"Magnate naviero envió 150.000 barriles de gasolina a Venezuela\n\nEl envío que hizo la compañía de Ruperti, Maroil Trading, llegó el viernes por la noche en el tanquero Aldan al terminal de la refinería El Palito\n\nVenezuela recibió un envío de 150.000 barriles de gasolina de una compañía propiedad del magnate naviero Wilmer Ruperti, en momentos que la nación miembro de la OPEP enfrenta la peor escasez de combustible en décadas.\n\nEl envío que hizo la compañía de Ruperti, Maroil Trading, llegó el viernes por la noche en el tanquero Aldan al terminal de la refinería El Palito, en la costa..."
0,https://es.mercopress.com/2020/04/29/oit-preve-perdida-de-300-millones-de-empleos-en-america-latina-debido-al-covid-19,south_america,mercopress,OIT prevé pérdida de 300 millones de empleos en América Latina debido al covid-19,2020-04-29,"OIT prevé pérdida de 300 millones de empleos en América Latina debido al covid-19\n\nLa OIT pidió medidas urgentes y flexibles para apoyar a los trabajadores y las empresas\n\nUn informe de la Organización Internacional del Trabajo (OIT) publicado esta semana indica que América Latina será la región más afectada desde el punto de vista laboral como resultado de la

In [65]:
df.to_csv(csv_name)


In [66]:
if(os.path.exists(output_path)):
    os.remove(output_path)
    print(('unprocessed file deleted'))

unprocessed file deleted
